<a href="https://colab.research.google.com/github/RoomRooms/2023BigData/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 

fns = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        fns.append(os.path.join(dirname, filename))
print(fns)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score, accuracy_score
from fastcore.basics import *
from fastcore.parallel import *
from os import cpu_count
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
df = pd.concat(objs=[
    pd.read_parquet(f) for f in fns
], copy=False, sort=False)
df.shape, df.columns

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        ###############################
        ########## 문제 3-1 ############

        self.conv1_1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(32)
        self.conv1_2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn1_2 = nn.BatchNorm2d(64)

        self.conv2_1 = nn.Conv2d(64, 94, kernel_size=3, stride=1, padding=1)
        self.bn2_1 = nn.BatchNorm2d(94)
        self.conv2_2 = nn.Conv2d(94, 128, kernel_size=3, stride=1, padding=1)
        self.bn2_2 = nn.BatchNorm2d(128)

        self.conv3_1 = nn.Conv2d(128, 192, kernel_size=3, stride=1, padding=1)
        self.bn3_1 = nn.BatchNorm2d(192)
        self.conv3_2 = nn.Conv2d(192, 256, kernel_size=3, stride=1, padding=1)
        self.bn3_2 = nn.BatchNorm2d(256)

        self.conv4_1 = nn.Conv2d(256, 320, kernel_size=3, stride=1, padding=1)
        self.bn4_1 = nn.BatchNorm2d(320)
        self.conv4_2 = nn.Conv2d(320, 384, kernel_size=3, stride=1, padding=1)
        self.bn4_2 = nn.BatchNorm2d(384)
        self.conv4_3 = nn.Conv2d(384, 448, kernel_size=3, stride=1, padding=1)
        self.bn4_3 = nn.BatchNorm2d(448)
        self.conv4_4 = nn.Conv2d(448, 512, kernel_size=3, stride=1, padding=1)
        self.bn4_4 = nn.BatchNorm2d(512)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512 * 1 * 1, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 10)

        ###############################
    def forward(self, x):
        x = F.relu(self.bn1_1(self.conv1_1(x)))
        x = F.relu(self.pool(self.bn1_2(self.conv1_2(x))))

        x = F.relu(self.bn2_1(self.conv2_1(x)))
        x = F.relu(self.pool(self.bn2_2(self.conv2_2(x))))

        x = F.relu(self.bn3_1(self.conv3_1(x)))
        x = F.relu(self.pool(self.bn3_2(self.conv3_2(x))))

        x = F.relu(self.bn4_1(self.conv4_1(x)))
        x = F.relu(self.bn4_2(self.conv4_2(x)))
        x = F.relu(self.bn4_3(self.conv4_3(x)))
        x = F.relu(self.pool(self.bn4_4(self.conv4_4(x))))
        
        x = torch.nn.Flatten()(x) # flatten all dimensions except batch

        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)

        x = self.fc3(x)
        return x

net = CNN()

In [ ]:
df_test = df.sample(frac=0.2, replace = False)
df_train = df.drop(index = df_test.index)
df_train.shape, df_test.shape

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)

net = net.to(device) # device로 Network 전송
print(net) # 모델 구조 확인

In [ ]:
batch_size = 16

In [ ]:
train_loader = torch.utils.data.DataLoader(df_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(df_test, batch_size=batch_size, shuffle=True)

In [ ]:
for X, y in train_loader:
    print(X, y)

In [ ]:
# training
EPOCHS = 20
for epoch in range(EPOCHS): # epochs수만큼 반복
    avg_loss = 0
    for data, target in train_loader:
        # print(data.shape)
        data = data.cuda() # 데이터도 gpu로 보내야함
        target = target.cuda()
        pred = net(data) # 
        # print(pred)
        optimizer.zero_grad() 
        loss = criterion(pred, target) # Model output과 target 비교
        loss.backward() # gradient 계산
        optimizer.step() # parameter 업데이트
        avg_loss += loss / len(train_loader) 
    print('[Epoch: {:>4}] loss = {:>.9}'.format(epoch + 1, avg_loss))

In [ ]:
print(df_train.index)

In [ ]:
print(train_loader.index)